In [51]:
import pandas as pd
import pickle
import gzip

In [52]:
from google.colab import files
uploaded = files.upload()

Saving seoul_populartime.pickle to seoul_populartime (1).pickle
Saving subway_xy.pickle to subway_xy (1).pickle


In [53]:
#장소랑 지하철 좌표 불러오기
with gzip.open('seoul_populartime.pickle','rb') as f:
    data = pickle.load(f)
with gzip.open('subway_xy.pickle','rb') as f:
    subway = pickle.load(f)

In [155]:
df1 = pd.DataFrame(data['aquarium'])

,id,name,address,types,coordinates,rating,rating_n,current_popularity,populartimes,time_spent,place
0,ChIJJ6PXkEOffDUROogCAw7CKtE,Aqua Planet 63,"50 63-ro, Yeoui-dong, Yeongdeungpo-gu","[aquarium, point_of_interest, establishment]","{'lat': 37.519286, 'lng': 126.9398004}",4.1,362,16,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[60, 90]",aquarium
1,ChIJ72K1TGqkfDURa_ENqoUeeXs,Coex Aquarium,"513 Yeongdong-daero, Gangnam-gu","[aquarium, tourist_attraction, point_of_intere...","{'lat': 37.513131, 'lng': 127.0582827}",4.3,7423,11,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[90, 150]",aquarium


In [156]:
# 장소 하나의 데이터프레임으로 만들기
places = list(data.keys())
df = pd.DataFrame()
for i in places:
  df1 = pd.DataFrame(data[i])
  df1['places'] = i
  df = pd.concat([df,df1])
df = df.reset_index(drop=True)

In [157]:
print(len(df),len(df.columns))

21531 12


In [158]:
# X, Y 좌표로 데이터 프레임 빼기
def preprocessing_data(df):
  df['lat'] = [df['coordinates'][i]['lat'] for i in range(len(df))]
  df['lng'] = [df['coordinates'][i]['lng'] for i in range(len(df))]
  return df
df = preprocessing_data(df)

In [187]:
#station: 지하철역, place: 카테고리
def listup(df, station, place = 'all', k=0.01):
  input_coordinates = subway[station]
  df['lat_dis'] = abs(df['lat'] - input_coordinates[0])
  df['lng_dis'] = abs(df['lng'] - input_coordinates[1])
  df1 = df.query("lat_dis < {} and lng_dis < {}".format(k,k))
  if place == 'all':
    return df1
  else:
    df1 = df1[df1['places']== place]
  return df1

In [189]:
kangnam = listup(df,'강남')
len(kangnam)

803

In [190]:
kangnam = listup(df,'강남','cafe')
len(kangnam)

120